In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models

import sys
sys.path.append('..')
from utils import preprocess, get_windows

In [3]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [4]:
nlp = spacy.load('en')

# Load dataset

In [5]:
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
docs = dataset['data']

In [6]:
# number of documents
len(docs)

18846

In [7]:
# store an index with a doc
docs = [(i, doc) for i, doc in enumerate(docs)]

# Preprocess dataset and create windows

In [8]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

100%|██████████| 18846/18846 [00:48<00:00, 391.67it/s]


number of removed short documents: 3985
total number of tokens: 1439861
number of tokens to be removed: 393091
number of additionally removed short documents: 2032
total number of tokens: 1023189

minimum word count number: 14
this number can be less than MIN_COUNTS because of document removal


In [9]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [10]:
data = []
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

12829it [00:03, 4216.57it/s]


In [11]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [12]:
# number of windows (equals to the total number of tokens)
data.shape[0]

1023189

# Get unigram distribution

In [13]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [14]:
vocab_size = len(decoder)
embedding_dim = 64

word_vectors = np.random.normal(
    0.0, np.sqrt(1.0/embedding_dim), 
    size=(vocab_size, embedding_dim)
).astype('float32')

# number of unique words
vocab_size

7460

# Prepare initialization for document weights

In [15]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [16]:
%%time
n_topics = 20
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

CPU times: user 1min 22s, sys: 1.18 s, total: 1min 23s
Wall time: 45.5 s


In [17]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : myer player hit ground ball wire john fan radar play
topic 1 : car buy sell price bike gun sale pay engine lot
topic 2 : book stephanopoulos president church talk job vote consider revelation statement
topic 3 : team play win player hockey season gun firearm league nhl
topic 4 : server widget window copy client application sun function display library
topic 5 : bike father apartment woman daughter old motorcycle door iran live
topic 6 : homosexual sex homosexuality male sexual paul giz marriage sin disease
topic 7 : space turkey dro stanley disk voyager alarm commercial launcher quantum
topic 8 : space launch earth orbit nasa mission satellite moon solar planet
topic 9 : church public issue president school house general federal today attorney
topic 10 : medical patient doctor treatment cause disease pain cancer test medicine
topic 11 : jesus word san israel lose creator west yassin mormon text
topic 12 : child kill fire happen fbi leave home war woman away
topic 13 : claim b

In [18]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

100%|██████████| 12829/12829 [00:28<00:00, 456.76it/s]


In [19]:
x = doc_weights_init + np.random.uniform(high=0.1, size=doc_weights_init.shape)
doc_weights_init = np.log(x/x.sum(1, keepdims=True))

# Save data

In [20]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)